In [1]:
import tensorflow as tf
from numba import cuda
import random
import matplotlib.pyplot as plt

tf.keras.backend.clear_session()
%matplotlib inline


In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [3]:
pretrained_model = InceptionV3(
    input_shape=(150,150,3),
    include_top=False,
    weights=None
)

In [4]:
pretrained_model.load_weights('../data/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [5]:
pretrained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [5]:
for layer in pretrained_model.layers:
    layer.trainable = False

In [15]:
last_layer = pretrained_model.get_layer('mixed7')
last_output = last_layer.output

In [21]:
print(last_output)

Tensor("mixed7/Identity:0", shape=(None, 7, 7, 768), dtype=float32)


In [17]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs):
        if logs.get('acc') > 0.97:
            print("Reached 97% Accuracy so cancelling training!")
            self.model.stop_training =True

#### Below steps are not implemented as memory exhausted

In [27]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024,activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1,activation='sigmoid')(x)


ResourceExhaustedError: OOM when allocating tensor with shape[37632,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [ ]:
model = tf.keras.Model(pretrained_model,x)

In [ ]:
model.summary()

In [30]:
data_dir = '../data/horses-vs-humans/'
train_dir = '../data/horses-vs-humans/train/'
valid_dir = '../data/horses-vs-humans/validation/'
train_horses_dir = '../data/horses-vs-humans/train/horses/'
train_humans_dir = '../data/horses-vs-humans/train/humans/'
valid_horses_dir = '../data/horses-vs-humans/validation/horses/'
valid_humans_dir = '../data/horses-vs-humans/validation/humans/'

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [32]:
train_datagen = ImageDataGenerator(rescale= 1 / 255.0,
                                  vertical_flip = True,
                                  rotation_range=40,
                                  shear_range = 0.2,width_shift_range=0.2,zoom_range=0.2,fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale = 1 / 255.0)

In [33]:
batch_size = 16
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150,150),
                                                   batch_size=batch_size,
                                                   class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                   target_size=(150,150),
                                                   batch_size=batch_size,
                                                   class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [34]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',optimizer = RMSprop(lr=0.0001),metrics = ['acc'])

NameError: name 'model' is not defined

In [35]:
callback = MyCallback()

In [ ]:
history = model.fit_generator(train_generator,
                              validation_data=valid_generator,
                              steps_per_epoch=1027/batch_size,
                              epochs = 20,
                              validation_steps=256/batch_size,
                              shuffle=True,
                              verbose=1,
                             callbacks=[callbacks])

In [ ]:
from keras.preprocessing import image

path = '../data/horses-vs-humans/test/horse1.jpeg'
img = image.load_img(path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])
if classes[0]>0.5:
    print(path.split('/')[-1] + " is a human")
else:
    print(path.split('/')[-1] + " is a horse")


In [36]:
cuda.close()